# function call调取

- @作者：AFAN
- @时间：2025-03-23
- @B站：[AFAN的金融科技](https://space.bilibili.com/1020711810)
- @公众号：[AFAN Fintech（点击通过文章关注）](https://mp.weixin.qq.com/s/f1Om7j9jndkqhlyfyqrgOw)
- @微信：afan-life
- @github：https://github.com/AFAN-LIFE/AFAN-LIFE
- @知识星球（付费分享）：[AFAN的金融科技](https://wx.zsxq.com/group/88885818554422)
- *注：本分享来自于AFAN的公开分享，转载请注明出处

In [53]:
SK_CODE = '你的token'

# 函数定义

In [54]:
def get_current_weather(location: str, unit: str = "摄氏度"):
    """获取指定地点的当前天气信息
    
    参数:
        地点 (str): 城市名称，例如："北京"
        温度单位 (str): 温度单位，可选"摄氏度"或"华氏度"
    
    返回:
        dict: 包含天气信息的字典
    """
    weather_data = {
        "location": location,
        "temperature": "25",
        "unit": unit,
        "forecast": "sunny",
        "humidity": 65,
    }
    return weather_data

# 用户提问

In [55]:
messages = [
        {
            "role": "user",
            "content": "南京今天的天气如何？"
        }
]

# 函数注册

In [56]:
import requests

url = "https://api.siliconflow.cn/v1/chat/completions"

payload = {
    "model": "deepseek-ai/DeepSeek-V3",
    "messages": messages,
    "tools": [
        {
            'type': 'function',
            'function': {
                'name': 'get_current_weather',
                'description': '获取指定城市的当前天气信息，包括温度、天气状况和湿度等',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'location': {
                            'type': 'string',
                            'description': '城市名称，例如：北京、上海',
                        },
                        'unit': {
                            'type': 'string', 
                            'enum': ['摄氏度', '华氏度'],
                            'description': '温度显示使用的单位'
                        },
                    },
                    'required': ['location'],
                },
            }
        }
    ],
    "tool_choice": "auto",  # 自动选择是否调用工具
}

In [57]:
headers = {
    "Authorization": f"Bearer {SK_CODE}",
    "Content-Type": "application/json"
}

# 忽略 SSL 验证
response = requests.post(url, json=payload, headers=headers, verify=False)

D:\software\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


# 函数触发

In [58]:
response.text

'{"id":"0195d76c475baca627514137a92acbc9","object":"chat.completion","created":1743075953,"model":"deepseek-ai/DeepSeek-V3","choices":[{"index":0,"message":{"role":"assistant","content":"","tool_calls":[{"id":"0195d76c55607eadef10d16fa0550d64","type":"function","function":{"name":"get_current_weather","arguments":"{\\"location\\":\\"南京\\",\\"unit\\":\\"摄氏度\\"}"}}]},"finish_reason":"tool_calls"}],"usage":{"prompt_tokens":127,"completion_tokens":24,"total_tokens":151},"system_fingerprint":""}'

In [59]:
import json
content_res = json.loads(response.text)['choices'][0]['message']
content_res

{'role': 'assistant',
 'content': '',
 'tool_calls': [{'id': '0195d76c55607eadef10d16fa0550d64',
   'type': 'function',
   'function': {'name': 'get_current_weather',
    'arguments': '{"location":"南京","unit":"摄氏度"}'}}]}

# 函数调用

In [60]:
function_res = json.loads(response.text)['choices'][0]['message']['tool_calls'][0]['function']
function_res

{'name': 'get_current_weather', 'arguments': '{"location":"南京","unit":"摄氏度"}'}

In [61]:
function_out = eval(f"{function_res['name']}(**{function_res['arguments']})")
function_out

{'location': '南京',
 'temperature': '25',
 'unit': '摄氏度',
 'forecast': 'sunny',
 'humidity': 65}

# 结果润色

In [62]:
messages.append(content_res)
messages.append({
    'role': 'tool',
    'content': f'{function_out}',
    'tool_call_id': content_res['tool_calls'][0]['id']
})

In [63]:
messages

[{'role': 'user', 'content': '南京今天的天气如何？'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'id': '0195d76c55607eadef10d16fa0550d64',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location":"南京","unit":"摄氏度"}'}}]},
 {'role': 'tool',
  'content': "{'location': '南京', 'temperature': '25', 'unit': '摄氏度', 'forecast': 'sunny', 'humidity': 65}",
  'tool_call_id': '0195d76c55607eadef10d16fa0550d64'}]

In [64]:
import requests

url = "https://api.siliconflow.cn/v1/chat/completions"

payload = {
    "model": "deepseek-ai/DeepSeek-V3",
    "messages": messages,
    "tools": [
        {
            'type': 'function',
            'function': {
                'name': 'get_current_weather',
                'description': '获取指定城市的当前天气信息，包括温度、天气状况和湿度等',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'location': {
                            'type': 'string',
                            'description': '城市名称，例如：北京、上海',
                        },
                        'unit': {
                            'type': 'string', 
                            'enum': ['摄氏度', '华氏度'],
                            'description': '温度显示使用的单位'
                        },
                    },
                    'required': ['location'],
                },
            }
        }
    ],
    "tool_choice": "auto",  # 自动选择是否调用工具
}

In [65]:
headers = {
    "Authorization": f"Bearer {SK_CODE}",
    "Content-Type": "application/json"
}

# 忽略 SSL 验证
response = requests.post(url, json=payload, headers=headers, verify=False)

D:\software\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [66]:
json.loads(response.text)

{'id': '0195d76c57237349faa6c9b493e922bf',
 'object': 'chat.completion',
 'created': 1743075956,
 'model': 'deepseek-ai/DeepSeek-V3',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '南京今天的天气晴朗，温度为25摄氏度，湿度为65%。天气较为舒适，适合外出活动。'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 189, 'completion_tokens': 21, 'total_tokens': 210},
 'system_fingerprint': ''}